In [ ]:
import json
from typing import Any, Dict, List, Tuple

In [ ]:
def getNFTVolumes(
    st_block: int, end_block: int, chainID: int
) -> Tuple[Dict[str, Dict[str, float]], List[DataNFT]]:
    """
    @description
      Query the chain for datanft volumes within the given block range.

    @return
      nft_vols_at_chain -- dict of [basetoken_addr][nft_addr]:vol_amt
      NFTinfo -- list of DataNFT objects
    """
    print("getVolumes(): begin")

    NFTvols: Dict[str, Dict[str, float]] = {}
    NFTinfo_tmp: Dict[str, Dict[str, Dict[str, Any]]] = {}
    NFTinfo = []

    chunk_size = 1000  # max for subgraph = 1000
    offset = 0
    while True:
        query = """
        {
          orders(where: {block_gte:%s, block_lte:%s}, skip:%s, first:%s) {
            id,
            datatoken {
              id
              symbol
              nft {
                id
              }
            },
            lastPriceToken,
            lastPriceValue,
            block
          }
        }
        """ % (
            st_block,
            end_block,
            offset,
            chunk_size,
        )
        offset += chunk_size
        result = submitQuery(query, chainID)
        new_orders = result["data"]["orders"]
        if new_orders == []:
            break
        for order in new_orders:
            lastPriceValue = float(order["lastPriceValue"])
            if lastPriceValue == 0:
                continue
            nft_addr = order["datatoken"]["nft"]["id"].lower()
            basetoken_addr = order["lastPriceToken"]

            if basetoken_addr not in NFTvols:
                NFTvols[basetoken_addr] = {}

            if nft_addr not in NFTvols[basetoken_addr]:
                NFTvols[basetoken_addr][nft_addr] = 0.0
            NFTvols[basetoken_addr][nft_addr] += lastPriceValue

            ### Store nft symbol for later use
            if not basetoken_addr in NFTinfo_tmp:
                NFTinfo_tmp[basetoken_addr] = {}

            if not nft_addr in NFTinfo_tmp[basetoken_addr]:
                NFTinfo_tmp[basetoken_addr][nft_addr] = {}

            NFTinfo_tmp[basetoken_addr][nft_addr]["symbol"] = order["datatoken"][
                "symbol"
            ]

    for base_addr in NFTinfo_tmp:
        for nft_addr in NFTinfo_tmp[base_addr]:
            datanft = DataNFT(
                nft_addr,
                chainID,
                NFTinfo_tmp[base_addr][nft_addr]["symbol"],
                base_addr,
                NFTvols[base_addr][nft_addr],
            )
            NFTinfo.append(datanft)

    print("getVolumes(): done")
    return NFTvols, NFTinfo
